# 1. Data reading and preprocessing
 

## 1.1 imports

In [2]:
!pip install unidecode
!pip install pytorch-pretrained-bert
!pip install pytorch-transformers
!pip install politenessr
!pip install stanza
import numpy as np

import json
#import convokit
#from convokit.politenessStrategies.politenessStrategies import PolitenessStrategies

import nltk
import pandas as pd
!python -m spacy download en
#from convokit import Corpus, download
import pickle
import sklearn
#from convokit.classifier.classifier import Classifier
#from convokit import TextParser
nltk.download('punkt')
import stanza as stanza
stanza.download('en')
from nltk.tokenize.punkt import PunktSentenceTokenizer
from spacy.lang.en import English
from  tqdm.notebook import tqdm
!pip3 install -U scikit-learn

import joblib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/cormak/.local/lib/python3.8/site-packages/en_core_web_sm -->
/home/cormak/.local/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
[nltk_data] Downloading package punkt to /home/cormak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2020-12-09 15:05:20 INFO: Downloading default packages for language: en (English)...
2020-12-09 15:05:21 INFO: File e

In [3]:
# what do you want to do ?
REDO_POLITENESS=True
REDO_SENTIMENTS=True
GOOGLE_COLAB = False
if REDO_POLITENESS and REDO_SENTIMENTS:
  import json
  import requests
  # Combine all datasets int one here
  url_train = "https://raw.githubusercontent.com/DenisPeskov/2020_acl_diplomacy/master/data/train.jsonl"
  url_test = "https://raw.githubusercontent.com/DenisPeskov/2020_acl_diplomacy/master/data/test.jsonl"
  url_validation = "https://raw.githubusercontent.com/DenisPeskov/2020_acl_diplomacy/master/data/validation.jsonl"
  response_train = requests.get(url_train)
  response_test = requests.get(url_test)
  response_validation = requests.get(url_validation)
  deception_train= [json.loads(jline) for jline in response_train.content.splitlines()]
  deception_test= [json.loads(jline) for jline in response_test.content.splitlines()]
  deception_validation= [json.loads(jline) for jline in response_validation.content.splitlines()]
  print(len(deception_train))
  print(len(deception_test))
  print(len(deception_validation))
  deception_validation.extend(deception_test)
  deception_validation.extend(deception_train)
  deception=deception_validation
  print(len(deception))


  if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    with open('/content/drive/MyDrive/ada-2020-project-milestone-p3-p3_data-bayes/deception_wthout_politeness_sentiment.pkl', 'wb') as _deception:  
      deception = pickle.dump(deception, _deception)
  else:
    with open('deception_wthout_politeness_sentiment.pkl', 'wb') as _deception:
      pickle.dump(deception, _deception)
else:
  if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    with open('/content/drive/MyDrive/ada-2020-project-milestone-p3-p3_data-bayes/preprocessed_deception.pkl', 'rb') as _deception:  
      deception = pickle.load(_deception)
  else:
    with open('./preprocessed_deception.pkl', 'rb') as preprocessed_deception:  
      deception = pickle.load(preprocessed_deception)



189
42
21
252


## 1.2 politeness

As the original politeness classifier used for the first paper was a bit outdated and not really maintainable we decided to go for another classifier 

In [4]:
if REDO_POLITENESS:
    from politenessr import Politenessr
    pr = Politenessr(CPU_COUNT=12, CHUNKSIZE=120)

12/09/2020 15:05:31 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /home/cormak/.cache/torch/pytorch_transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
12/09/2020 15:05:31 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_voc

In [5]:
if REDO_POLITENESS:
  for index , game in tqdm( enumerate(deception)):
    politeness=[]
    for message in tqdm(game['messages']):
      politeness.append(pr.predict([message]))#np.mean(nltk.tokenize.sent_tokenize(message)))
    deception[index]['politeness']=politeness

12/09/2020 15:05:34 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/cormak/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


100%|██████████| 1/1 [00:00<00:00, 31.56it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.94it/s]
12/09/2020 15:05:36 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/cormak/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


100%|██████████| 1/1 [00:00<00:00, 43.19it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  3.56it/s]
12/09/2020 15:05:37 - 

KeyboardInterrupt: 

In [ ]:
if REDO_SENTIMENTS:
  if GOOGLE_COLAB:
    with open('/content/drive/MyDrive/ada-2020-project-milestone-p3-p3_data-bayes/preprocessed_deception.pkl', 'wb') as preprocessed_deception:
      pickle.dump(deception, preprocessed_deception)
  else:  
    with open('./preprocessed_deception.pkl', 'wb') as preprocessed_deception:
      pickle.dump(deception, preprocessed_deception)

* creation of features for politeness

## 1.3 sentiments


In [ ]:
if REDO_SENTIMENTS:
  nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')
  for index, game in tqdm(enumerate(deception)):
      deception[index]['positive_sentiments']=[]
      deception[index]['negative_sentiments']=[]
      deception[index]['neutral_sentiments']=[]
      for message in game['messages']:
        counts=[0,0,0]
        message=nlp(message)
        for sentence in message.sentences:
          counts[sentence.sentiment]+=1
        deception[index]['negative_sentiments'].append(counts[0])
        deception[index]['neutral_sentiments'].append(counts[1])
        deception[index]['positive_sentiments'].append(counts[2])

In [ ]:
if REDO_SENTIMENTS:
  if GOOGLE_COLAB:
    with open('/content/drive/MyDrive/ada-2020-project-milestone-p3-p3_data-bayes/preprocessed_deception.pkl', 'wb') as preprocessed_deception:
      pickle.dump(deception, preprocessed_deception)
  else:  
    with open('./preprocessed_deception.pkl', 'wb') as preprocessed_deception:
      pickle.dump(deception, preprocessed_deception)